#**PLN - Universidad Nacional de Colombia**

##Taller 1
###Procesamiento: NLTK,Spacy,Tokenización, normlización, Expresiones Regulares
###Profesora Elizabeth León Guzmán
###Estudiante Francisco J. Salamanca
---

### 1. Cargar archivo
 Cargar el archivo twitter.tsv trabajado en clase (notebook 2)

In [13]:
import pandas as pd
#Cargar archivo
data = pd.read_csv("twitter.tsv", delimiter = "\t", quoting = 3)
del data["Unnamed: 0"]

In [14]:
#Ver los primeros 5 registros
data.head()

,id_tweet,created_at,user_name,text
0,1046764647116951553,2018-10-01 09:11:54,dpazmino,#lamadre para esos usuarios de @TransMilenio q...
1,1046833427301978112,2018-10-01 13:45:12,Citytv,8 años de un patio taller del SITP 'provisiona...
2,1046876307110412289,2018-10-01 16:35:36,RodrigoMutis,La @CAR_Cundi de manera antidemocrática y arbi...
3,1046904276692426754,2018-10-01 18:26:44,Luigi32H,@TransMilenio manden buses acaba de llegar uno...
4,1047149277330460673,2018-10-02 10:40:17,dhidalgo65,.@Bogota sigue disminuyendo el número de muert...


In [15]:
#Ver los ultimos 8 registros
!head -8 twitter.tsv

	id_tweet	created_at	user_name	text
0	1046764647116951553	2018-10-01 09:11:54	dpazmino	#lamadre para esos usuarios de @TransMilenio que cuando llevan morral no lo ponen en la parte de adelante y hacen estorbo #BogotáHumana
1	1046833427301978112	2018-10-01 13:45:12	Citytv	8 años de un patio taller del SITP 'provisional' ►https://t.co/l9BvjSUTI2 https://t.co/NCVKTkaeTE
2	1046876307110412289	2018-10-01 16:35:36	RodrigoMutis	La @CAR_Cundi de manera antidemocrática y arbitraria le impide a la @ReservaVDHammen hacer parte de la discusión sobre la Reserva van der Hammen, como vocero de la veeduría pido solidaridad para denunciar tal atropello
3	1046904276692426754	2018-10-01 18:26:44	Luigi32H	@TransMilenio manden buses acaba de llegar uno pero lleno. Otra media hora?
4	1047149277330460673	2018-10-02 10:40:17	dhidalgo65	.@Bogota sigue disminuyendo el número de muertes en el tráfico (4%  enero-agosto 2018 respecto al mismo periodo de 2017)Importante reducción en peatones Estabilidad en motocicl

###2. Tokenizar
Implementar una función para tokenizar considerando los siguientes casos:

*   Dividir tokens por espacios, “tab”, y “enters”

  Ej: ' this is a tweet ' -> 'this', 'is', 'a', 'tweet
*   Dividir por emoticones:

  Ej:  haha 😂😂! -> 'haha', '😂😂', '!'


*   NO dividir hashtags

    Ej: #don'tSplit
*   Separar signos de puntuación del texto (tokesn individuales):

    Ej: 'hey!' -> 'hey', '!'


*  Separar “Contractions” usando el “apostrophe”:

  Ej: "that's fine" -> 'that', "'s", 'fine'

  Ej: "don't do that" -> 'do', "n't", 'do', 'that'

In [26]:
import nltk
import re

def tokenize(text):
  # Define a regex pattern to match emojis
  emoji_pattern = re.compile(
      "["
      "\U0001F600-\U0001F64F"  # emoticons
      "\U0001F300-\U0001F5FF"  # symbols & pictographs
      "\U0001F680-\U0001F6FF"  # transport & map symbols
      "\U0001F1E0-\U0001F1FF"  # flags (iOS)
      "\U00002702-\U000027B0"
      "\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)

  # Replace emojis with spaces around them to ensure splitting
  text_with_spaces_around_emojis = emoji_pattern.sub(r' \g<0> ', text)

  # Pattern to not split hashtags, separate punctuation and contractions, and split by whitespace otherwise
  # This pattern prioritizes matching hashtags, then contractions, then punctuation (including quotes), and finally words or other non-whitespace characters.
  tokenizer_pattern = re.compile(r"(#\w+)|(\w+'\w+)|([.,!?;:\"'])|(\S+)")


  # The findall method returns a list of tuples, where each tuple contains the matches for the capturing groups.
  # We need to flatten this list of tuples into a single list of tokens, keeping only the non-empty matches.
  tokens = [item for sublist in tokenizer_pattern.findall(text_with_spaces_around_emojis) for item in sublist if item]

  # Add specific handling for splitting contractions like "don't" and "that's"
  # This is a post-processing step after the initial regex split
  processed_tokens = []
  for token in tokens:
      if "'" in token and re.match(r"\w+'\w+", token):
          # Split contractions like "that's" and "don't"
          match = re.match(r"(\w+)'(\w+)", token)
          if match:
              processed_tokens.append(match.group(1))
              processed_tokens.append("'" + match.group(2))
          else:
               # Handle cases like "don't"
              match = re.match(r"(\w+)n't", token)
              if match:
                  processed_tokens.append(match.group(1))
                  processed_tokens.append("n't")
              else:
                   processed_tokens.append(token) # keep token if not a recognized contraction pattern
      else:
          processed_tokens.append(token)

  return processed_tokens

In [27]:
result = data['text'].apply(tokenize)
print(result)

0      [#lamadre, para, esos, usuarios, de, @TransMil...
1      [8, años, de, un, patio, taller, del, SITP, ',...
2      [La, @CAR_Cundi, de, manera, antidemocrática, ...
3      [@TransMilenio, manden, buses, acaba, de, lleg...
4      [., @Bogota, sigue, disminuyendo, el, número, ...
                             ...                        
995    [Por, fin, se, transformó, 😱😱😱, #AnimesDel1, 🤩...
996    [I, 'm, at, Centro, Comercial, @ParqueLaColina...
997    [2018,, gracias, 💙, en, Bogotá,, Colombia, htt...
998    [Good, Morning, en, Bogotá,, Colombia, https:/...
999    [Para, la, señora, @MariaFdaCabal, los, únicos...
Name: text, Length: 1000, dtype: object


In [28]:
result[950]

['Jajajajajajajajajajaja',
 'Jajajajajajajajajajaja',
 'jajajajajajaja',
 'Jajajajajajajajajajaja',
 'jajajajajajaja',
 'Jajajajajajajajajajaja',
 'Jajajajajajajajajajaja',
 'Jajajajajajajajajajaja',
 'Jajajajajajajajajajaja',
 'jajajajajajaja',
 'Jajajajajajajajajajaja',
 'https://t.co/dDHAAgGrvJ']

In [29]:
#Buscar si hay un "hahaha" o "jajaja"
def find_laughter_tokens(tokens):
  laughter_pattern = re.compile(r"(ha|ja)\1+")
  found_laughter = []
  for token in tokens:
    if laughter_pattern.fullmatch(token):
      found_laughter.append(token)
  return found_laughter

risa = result.apply(find_laughter_tokens)

# Print the entries that contain laughter tokens
for index, laughter_list in risa.items():
  if laughter_list:
    print(f"Row {index}: {laughter_list}")

Row 10: ['jajaja']
Row 168: ['jajajajajajaja']
Row 250: ['jajajaja']
Row 302: ['jajajaja']
Row 471: ['jajaja']
Row 507: ['jaja']
Row 946: ['jajaja']
Row 950: ['jajajajajajaja', 'jajajajajajaja', 'jajajajajajaja']
Row 957: ['jaja']


### 3.  Normalizar
Después de la “tokenización”, aplicar normalización. Implementar una función para normalizar tokens que tienen secuencia de letras repetidas, como:

*   hahahahaha -> haha
*   cooooool -> cool
*   !?!?!?!!!! -> !?!?!!
*   !!!!!????? -> !!??

Aplicar otras tareas de normalización, como:
- convertir tokens a minúscula, menos los hashtags y cuentas de usuario.
- Encontar fechas y dejar todas en un mismo formato (escoga el formato al que desea dejar la fecha)

In [30]:
import re
from datetime import datetime

def normalice(tokens):
  normalized_tokens = []

  #transformar a minusculas, excepto #+ y @+
  processed_case_tokens = []
  for token in tokens:
    if not (token.startswith('#') or token.startswith('@')):
      processed_case_tokens.append(token.lower())
    else:
      processed_case_tokens.append(token)

  for token in processed_case_tokens:
    # Buscar diferentes formatos de fechas que puedan haber
    date_formats = ["%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y", "%d/%m/%Y"]
    is_date = False
    for fmt in date_formats:
      try:
        date_obj = datetime.strptime(token, fmt)
        normalized_tokens.append(date_obj.strftime("%Y-%m-%d")) # Formato estandar
        is_date = True
        break
      except ValueError:
        pass

    if is_date:
        continue #Seguir a otro token

    # Reducir caracteres especiales, para ja y ha
    normalized_token = re.sub(r'(ha|ja)\1{2,}', r'\1\1', token)
    normalized_token = re.sub(r'(.)\1{2,}', r'\1\1', normalized_token) # Keep general single char repetition reduction
    normalized_tokens.append(normalized_token)

  return normalized_tokens

In [41]:
normalized_result = result.apply(normalice)
print(normalized_result)

0      [#lamadre, para, esos, usuarios, de, @TransMil...
1      [8, años, de, un, patio, taller, del, sitp, ',...
2      [la, @CAR_Cundi, de, manera, antidemocrática, ...
3      [@TransMilenio, manden, buses, acaba, de, lleg...
4      [., @Bogota, sigue, disminuyendo, el, número, ...
                             ...                        
995    [por, fin, se, transformó, 😱😱, #AnimesDel1, 🤩,...
996    [i, 'm, at, centro, comercial, @ParqueLaColina...
997    [2018,, gracias, 💙, en, bogotá,, colombia, htt...
998    [good, morning, en, bogotá,, colombia, https:/...
999    [para, la, señora, @MariaFdaCabal, los, únicos...
Name: text, Length: 1000, dtype: object


In [42]:
normalized_result[950]

['jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'jaja',
 'https://t.co/ddhaaggrvj']

###4. Lematizacion- Steamming: Porter's Algorithm
- Seleccionar un documento en inglés y aplicar el algoritmo de Porter (si desea lo puede implementar). ¿Cuáles reglas aplicó?
- Construir un lematizador y stemming para español con al menos 10 reglas. Aplicarlo al conjunto de twitter.

#### Algoritmo de Porter

In [ ]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
#Leer e importar un archivo pdf (Ej. Paper)

import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
        return text
    except FileNotFoundError:
        return "Error: The file was not found."
    except Exception as e:
        return f"An error occurred: {e}"

pdf_text = extract_text_from_pdf('GenomeGenerative.pdf')
print(pdf_text[50000:55000])

 bioRxiv preprint (A) We used Evo 2 to generate chromosome- and genome-scale sequences using unconstrained autoregressive
generation. We prompted the model with portions of the H. sapiens mitochondrial genome, M. genitalium
genome, and S. cerevisiae chromosome III and generate DNA sequences with similar lengths as those of the
native sequence. (B) We prompted Evo 2 with genomic context as well as a portion of a highly conserved pro-
tein and generate, measuring the sequence recovery of the Evo 2 generated gene against the natural gene. (C)
Predicted rRNA, CDS, and tRNA counts in Evo 2 generated mitochondrial sequences using MitoZ compared
with the natural H. sapiens mitochondrial genome values. (D) Query cover versus sequence identity of gen-
erated mitochondrial sequences against nucleotide BLAST hits in the core_nt database with expect threshold
of 0.05. (E) Visualizations of Evo 2 generated sequences when prompted with a 3 kb sequence from the H.
sapiensmitochondrial genome, demonst

In [ ]:
#Tokenizar las palabras con  la funcion creada anteriormente
tokenized_pdf = tokenize(pdf_text)
print(tokenized_pdf[10000:20000])

['large', 'models', 'spanning', 'application', 'domains', 'of', 'biology', 'and', 'natural', 'language.', 'We', 'include', 'Pythia', 'Large', '(Biderman', 'et', 'al.,', '2023),', 'OLMo', '2', 'Large', '(Team', 'OLMo', 'et', 'al.,', '2024),', 'Evo', '1,', 'Falcon', '1', '180B', 'Almazrouei', 'et', 'al.', '(2023),', 'StarCoder', '2', '(Lozhkov', 'et', 'al.,', '2024),', 'DeepSeek', 'V3', '(Liu', 'et', 'al.,', '2024),', 'Llama', '3.1', 'Large', '(Dubey', 'et', 'al.,', '2024),', 'ESM', '3', 'Large', '(Hayes', 'et', 'al.,', '2025)', 'and', 'xTrimo', '(Chen', 'et', 'al.,', '2024).', 'Model', 'Model', 'Size', 'Tokens', 'FLOPS', 'est.', 'Fully', 'open:', 'data,', 'infrastructure,', 'weights', 'Evo', '2', '40B', '40.3B', '9.3T', '2.25×1024', 'Pythia', 'Large', '12B', '300B', '2.16×1022', 'OLMo', '2', 'Large', '13B', '5.6T', '4.37×1023', 'Open', 'data,', 'weights', 'Evo', '1', '7B', '300B', '1.26×1022', 'Falcon', '180B', '180B', '3.5T', '3.78×1024', 'StarCoder', '2', '15B', '15B', '4.3T', '3.87×1

In [ ]:
#implementacion de Algoritmo de porter

from nltk.stem.porter import *
p_stemmer = PorterStemmer()
words = tokenized_pdf
#Visualizar las primeras 100 tokens
for word in words[8000:8100]:
  print(word+" "+p_stemmer.stem(word))

accessibility access
in in
some some
genomic genom
regions region
and and
minimize minim
accessibility access
in in
other other
regions. regions.
AlthoughEvo2hasnoexplicitmechanismforconditioningitsgenerationsbasedonepigeneticstate,several althoughevo2hasnoexplicitmechanismforconditioningitsgenerationsbasedonepigeneticstate,sever
models model
such such
as as
Enformer enform
(Avsec (avsec
et et
al., al.,
2021) 2021)
and and
Borzoi borzoi
(Linder (linder
et et
al., al.,
2025) 2025)
demonstrate demonstr
accurate, accurate,
held-out held-out
performance perform
in in
predicting predict
chromatin chromatin
accessibility access
profiles profil
from from
DNA dna
sequence sequenc
across across
cell cell
types type
from from
human human
and and
mouse. mouse.
However, however,
models model
like like
Enformer enform
and and
Borzoi borzoi
are are
not not
trained train
as as
generative gener
models model
and and
assume assum
that that
their their
sequence sequenc
inputs input
come come
from from
na

Las Reglas aplicadas en el algoritmo de porter son:

- Regla 1:

Remover sufijos simples, como los de los plurales y verbos irregulares en pasado

("s", "es", "ed", "ing").


- Regla 2:

Remover sufijos complejos como -ational e -ization

- Regla 3:

Sufijos comunes son removidos como "er", "ly", "ment". "ness"

- Regla 4:

Sufijos comunes son removidos como “al”, “ance”, “ence”, etc.

- Regla 5

Reglas adicionales como remocion del la "e"



#### Lemmatizador y Stemming para el idioma Español

- Construir un lematizador y stemming para español con al menos 10 reglas. Aplicarlo al conjunto de twitter.

In [69]:
import re
import unicodedata

def lematizar_y_stem(normalized_tokens):
    """
    Función que recibe una lista de tokens normalizados en español
    y devuelve una lista de diccionarios con el token original y su stem.
    """

    resultados = []

    #Diccionario de algunos lemas irregulares
    lemas_irregulares = {
        "fui": "ir",
        "fuiste": "ir",
        "fue": "ir",
        "fueron": "ir",
        "soy": "ser",
        "eres": "ser",
        "es": "ser",
        "somos": "ser",
        "son": "ser",
        "quiero": "querer",
        "quieres": "querer",
        "quiere": "querer",
        "quieren": "querer"
    }

    for token in normalized_tokens:
        # Skip tokens starting with '#' or '@'
        if token.startswith('#') or token.startswith('@'):
            resultados.append({
                token # Return the token as is
            })
            continue # Move to the next token

        palabra = token.lower()

        # Eliminar tildes (already done in normalice, but keep for robustness)
        palabra = unicodedata.normalize('NFD', palabra).encode('ascii', 'ignore').decode('utf-8')

        stem = palabra  # Initialize stem with the processed word

        # Reglas de lematización (check for irregulars first)
        if palabra in lemas_irregulares:
            lemma = lemas_irregulares[palabra]
            stem = lemma # Use lemma as stem for irregular verbs for simplicity in this function

        else:
            # Reglas de stemming for regular words
            #Retirar sufijo ando
            if palabra.endswith("ando") or palabra.endswith("iendo"):
                stem = palabra[:-4]
            #Retirar sifijo mente
            elif palabra.endswith("mente"):
                stem = palabra[:-5]
            #Retirar sufijo itos e itas
            elif palabra.endswith("itos") or palabra.endswith("itas"):
                stem = palabra[:-4]
            #Retirar isima e isimo
            elif palabra.endswith("isima") or palabra.endswith("isimo"):
                stem = palabra[:-4]
            #Retirar sufijo ito e ita
            elif palabra.endswith("ito") or palabra.endswith("ita"):
                stem = palabra[:-3]
            #Retirar sufijo os y as
            elif palabra.endswith("os") or palabra.endswith("as"):
                stem = palabra[:-2]
            #Retirar sufijo plural es
            elif palabra.endswith("es"):
                stem = palabra[:-2]
            #Retirar sufijo plural s
            elif palabra.endswith("s") and len(palabra) > 3:
                stem = palabra[:-1]
            # No specific rule matched, stem is the lowercased word without accents


        resultados.append({
            token
        })

    return resultados


# Prueba with a list of normalized tokens
normalized_tokens_example = ["comiendo", "gatitos", "rapidamente", "ponerse", "fui", "quiero", "grandisima", "antidemocratica", "#hashtag", "@user", "bogota"]
res = lematizar_y_stem(normalized_tokens_example)

for r in res:
    print(r)

{'comiendo'}
{'gatitos'}
{'rapidamente'}
{'ponerse'}
{'fui'}
{'quiero'}
{'grandisima'}
{'antidemocratica'}
{'#hashtag'}
{'@user'}
{'bogota'}


In [70]:
lematized_normalized_ES = normalized_result.apply(lematizar_y_stem)
print(lematized_normalized_ES)

0      [{#lamadre}, {para}, {esos}, {usuarios}, {de},...
1      [{8}, {años}, {de}, {un}, {patio}, {taller}, {...
2      [{la}, {@CAR_Cundi}, {de}, {manera}, {antidemo...
3      [{@TransMilenio}, {manden}, {buses}, {acaba}, ...
4      [{.}, {@Bogota}, {sigue}, {disminuyendo}, {el}...
                             ...                        
995    [{por}, {fin}, {se}, {transformó}, {😱😱}, {#Ani...
996    [{i}, {'m}, {at}, {centro}, {comercial}, {@Par...
997    [{2018,}, {gracias}, {💙}, {en}, {bogotá,}, {co...
998    [{good}, {morning}, {en}, {bogotá,}, {colombia...
999    [{para}, {la}, {señora}, {@MariaFdaCabal}, {lo...
Name: text, Length: 1000, dtype: object


In [72]:
lematized_normalized_ES[951]


[{'ruta'},
 {'56b'},
 {'cortijo'},
 {'↔'},
 {'chapinero'},
 {'de'},
 {'engativa'},
 {'a'},
 {'chapinero'},
 {'(con'},
 {'mapas)'},
 {'-'},
 {'subete'},
 {'al'},
 {'#SITP'},
 {'https://t.co/sm32te5kus'}]